In [107]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

print(os.listdir("./images/"))

['pand_1.jpg', 'pand_10.jpg', 'pand_100.jpg', 'pand_100_2.jpg', 'pand_101.jpg', 'pand_101_2.jpg', 'pand_102.jpg', 'pand_102_2.jpg', 'pand_103.jpg', 'pand_103_2.jpg', 'pand_104.jpg', 'pand_104_2.jpg', 'pand_105.jpg', 'pand_105_2.jpg', 'pand_106.jpg', 'pand_106_2.jpg', 'pand_107.jpg', 'pand_107_2.jpg', 'pand_108.jpg', 'pand_108_2.jpg', 'pand_109.jpg', 'pand_109_2.jpg', 'pand_10_2.jpg', 'pand_11.jpg', 'pand_110.jpg', 'pand_110_2.jpg', 'pand_111.jpg', 'pand_111_2.jpg', 'pand_112.jpg', 'pand_112_2.jpg', 'pand_113.jpg', 'pand_113_2.jpg', 'pand_114.jpg', 'pand_114_2.jpg', 'pand_115.jpg', 'pand_115_2.jpg', 'pand_116.jpg', 'pand_116_2.jpg', 'pand_117.jpg', 'pand_117_2.jpg', 'pand_118.jpg', 'pand_118_2.jpg', 'pand_119.jpg', 'pand_119_2.jpg', 'pand_11_2.jpg', 'pand_12.jpg', 'pand_120.jpg', 'pand_120_2.jpg', 'pand_121.jpg', 'pand_121_2.jpg', 'pand_122.jpg', 'pand_122_2.jpg', 'pand_123.jpg', 'pand_123_2.jpg', 'pand_124.jpg', 'pand_124_2.jpg', 'pand_125.jpg', 'pand_125_2.jpg', 'pand_126.jpg', 'pand_

In [135]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os
def load_house_attributes(inputPath):
    # aanmaken dataset van .csv data (Immoweb) + toekennen kolomnamen
    colnames = ["type","links-href","prijs","slaapkamers","bewoonbare_ruimte","grond","bouwjaar","conditie","badkamers","energieklasse","postcode","image1","image2"]
    df = pd.read_csv(inputPath, sep=";", header=None, names=colnames, skiprows=1)

    return df

def process_house_attributes(df, train, test):
    # initialisatie van continue data (meetbaar)
    continue_features = ["slaapkamer", "badkamer", "bewoonbare_ruimte"]

    # toepassen min-max scaling op continue features (range 0-1)
    cs = MinMaxScaler()
    trainContinue = cs.fit_transform(train[continue_features])
    testContinue = cs.transform(test[continue_features])

    # one-hot encode the zip code categorical data (by definition of
    # one-hot encoing, all output features are now in the range [0, 1])
    zipBinarizer = LabelBinarizer().fit(df["postcode"])
    trainDiscrete = zipBinarizer.transform(train["postcode"])
    testDiscrete = zipBinarizer.transform(test["postcode"])

    # concatenatie van de discrete en continue features -> training en testing set
    trainX = np.hstack([trainContinue, trainDiscrete])
    testX = np.hstack([testContinue, testDiscrete])

# return the concatenated training and testing data
    return (trainX, testX)

def load_house_images(df, inputPath):

    images = []
    for i in df.index.values:
        # sorteren van foto's in juiste volgorde per pand
        prefix =  "/pand_{}".format(i + 1)
        basePath = os.path.sep.join([inputPath, prefix])
        print(basePath)
        housePaths = sorted([basePath + ".jpg"] + glob.glob(basePath + "_*"))
        # creatie matrix die image data van huizen zal bevatten.
        inputImages = []
        outputImage = np.zeros((64, 32, 3), dtype="uint8")

        # loop over the input house paths
        for housePath in housePaths:
            print(housePath)
            # load the input image, resize it to be 32 32, and then
            # update the list of input images
            image = cv2.imread(housePath)
            image = cv2.resize(image, (32, 32))
            inputImages.append(image)
        # afbeeldingen van panden (per 2) boven elkaar plaatsen zodat die als geheel door het model getrained kunnen worden
        outputImage[0:32, 0:32] = inputImages[0]
        outputImage[32:64, 0:32] = inputImages[1]


        # toevoegen van alle tiled images in array waarop model getrained zal worden
        images.append(outputImage)
        

        #from matplotlib import pyplot as plt
        #plt.imshow(images[0], interpolation='nearest')
        #plt.show()
        #break
    
    
    return np.array(images)

In [136]:
#from tensorflow.keras.optimizers import Adam
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
import os

# inladen van data uit csv bestand om dataset op te stellen
print("[INFO] loading house attributes...")
inputPath = "immoweb_listings.csv"
df = load_house_attributes(inputPath)

#converteren string labels to integer values
df["conditie"] = df["conditie"].map({"Te renoveren": 1, "Op te frissen": 2, "Goed" : 3, "Net gerenoveerd" : 4, "Uitstekende staat" : 5})

print(df)

[INFO] loading house attributes...
          type                                         links-href      prijs  \
0         Huis  https://www.immoweb.be/nl/zoekertje/huis/te-ko...  € 265.000   
1         Huis  https://www.immoweb.be/nl/zoekertje/huis/te-ko...  € 149.000   
2         Huis  https://www.immoweb.be/nl/zoekertje/huis/te-ko...  € 175.000   
3    Boerderij  https://www.immoweb.be/nl/zoekertje/boerderij/...  € 149.000   
4         Huis  https://www.immoweb.be/nl/zoekertje/huis/te-ko...  € 370.000   
..         ...                                                ...        ...   
532       Huis  https://www.immoweb.be/nl/zoekertje/huis/te-ko...  € 499.000   
533       Huis  https://www.immoweb.be/nl/zoekertje/huis/te-ko...  € 445.000   
534       Huis  https://www.immoweb.be/nl/zoekertje/huis/te-ko...  € 388.000   
535       Huis  https://www.immoweb.be/nl/zoekertje/huis/te-ko...  € 136.000   
536       Huis  https://www.immoweb.be/nl/zoekertje/huis/te-ko...  € 149.000   

    

In [137]:
# normaliseren van alle afbeeldingen in een [0-1] range
print("inladen foto's huizen")
images = load_house_images(df, "./images")
images = images / 255.0

%matplotlib inline  


inladen foto's huizen
./images\/pand_1
./images\/pand_1.jpg
./images\pand_1_2.jpg
./images\/pand_2
./images\/pand_2.jpg
./images\pand_2_2.jpg
./images\/pand_3
./images\/pand_3.jpg
./images\pand_3_2.jpg
./images\/pand_4
./images\/pand_4.jpg
./images\pand_4_2.jpg
./images\/pand_5
./images\/pand_5.jpg
./images\pand_5_2.jpg
./images\/pand_6
./images\/pand_6.jpg
./images\pand_6_2.jpg
./images\/pand_7
./images\/pand_7.jpg
./images\pand_7_2.jpg
./images\/pand_8
./images\/pand_8.jpg
./images\pand_8_2.jpg
./images\/pand_9
./images\/pand_9.jpg
./images\pand_9_2.jpg
./images\/pand_10
./images\/pand_10.jpg
./images\pand_10_2.jpg
./images\/pand_11
./images\/pand_11.jpg
./images\pand_11_2.jpg
./images\/pand_12
./images\/pand_12.jpg
./images\pand_12_2.jpg
./images\/pand_13
./images\/pand_13.jpg
./images\pand_13_2.jpg
./images\/pand_14
./images\/pand_14.jpg
./images\pand_14_2.jpg
./images\/pand_15
./images\/pand_15.jpg
./images\pand_15_2.jpg
./images\/pand_16
./images\/pand_16.jpg
./images\pand_16_2.j

In [138]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
print(df.shape[0])
print(images.shape[0])
split = train_test_split(df, images, test_size=0.25, random_state=35)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

537
537


In [113]:
# import the necessary packages
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model

def create_mlp(dim, regress=False):
    # define our MLP network
    model = Sequential()
    model.add(Dense(8, input_dim=dim, activation="relu"))
    model.add(Dense(4, activation="relu"))

    # check to see if the regression node should be added
    if regress:
        model.add(Dense(1, activation="linear"))

    # return our model
    return model

def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False):
    # initialize the input shape and channel dimension, assuming
    # TensorFlow/channels-last ordering
    inputShape = (height, width, depth)
    chanDim = -1

    # define the model input
    inputs = Input(shape=inputShape)

    # loop over the number of filters
    for (i, f) in enumerate(filters):
        # if this is the first CONV layer then set the input
        # appropriately
        if i == 0:
            x = inputs

        # CONV => RELU => BN => POOL
        x = Conv2D(f, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)

    # flatten the volume, then FC => RELU => BN => DROPOUT
    x = Flatten()(x)
    x = Dense(16)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Dropout(0.5)(x)

    # apply another FC layer, this one to match the number of nodes
    # coming out of the MLP
    x = Dense(4)(x)
    x = Activation("relu")(x)

    # check to see if the regression node should be added
    if regress:
        x = Dense(1, activation="linear")(x)

    # construct the CNN
    model = Model(inputs, x)

    # return the CNN
    return model

ModuleNotFoundError: No module named 'tensorflow'